# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'data': {
        'd': 2, #degree
        'n': 9, #number of variables
        'monomial_vars': None, #int or None
        'laurent': True, #use Laurent polynomials (negative degree with up to -d)
        'neg_d': 0, #int or None
        'neg_d_prob': 0, #probability before other adjustments
        'sparsity': None,
        'sample_sparsity': None,
        'x_max': 1,
        'x_min': 0.05,
        'x_distrib': 'uniform', #'normal', 'uniform',
        'a_max': 1,
        'a_min': -1,
        'polynomial_data_size': 50000, #number of polynomials to generate
        'noise': 0.15, #fraction of mean(abs(y_data)) 
        'noise_distrib': 'normal', #'normal', 'uniform'
        
        'border_min': 0.2, #needs to be between 0 and (x_max-x_min)/2
        'border_max': 0.4,
        'lower_degree_prob': 0.5,
        'a_zero_prob': 0.25,
        'a_random_prob': 0.1,       
        
        'same_training_all_lambda_nets': False,
    },
    'lambda_net': {
        'lambda_dataset_size': 5000, #number of samples per polynomial
    },    
    'computation':{
        'n_jobs': 20,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}


In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['computation'])

## Imports

In [3]:
from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter

In [4]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n] 


os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers if use_gpu else ''

sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

sys.path.append('..')

In [5]:
from utilities.utility_functions import flatten, rec_gen

list_of_monomial_identifiers_extended = []

if laurent:
    variable_sets = [list(flatten([[_d for _d in range(d+1)], [-_d for _d in range(1, neg_d+1)]])) for _ in range(n)]
    list_of_monomial_identifiers_extended = rec_gen(variable_sets)    
        
    print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
    #print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(sparsity))
    #print('Sparsity:' + str(sparsity))
    print(list_of_monomial_identifiers_extended)        
else:
    variable_sets = [[_d for _d in range(d+1)] for _ in range(n)]  
    list_of_monomial_identifiers_extended = rec_gen(variable_sets)

    print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
    #print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(sparsity))
    #print('Sparsity: ' + str(sparsity))
    print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    if np.sum(monomial_identifier) <= d:
        if monomial_vars == None or len(list(filter(lambda x: x != 0, monomial_identifier))) <= monomial_vars:
            list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
#print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(sparsity))
#print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)


List length: 19683
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 2], [0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 2, 1], [0, 0, 0, 0, 0, 0, 0, 2, 2], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 2], [0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 2], [0, 0, 0, 0, 0, 0, 1, 2, 0], [0, 0, 0, 0, 0, 0, 1, 2, 1], [0, 0, 0, 0, 0, 0, 1, 2, 2], [0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 2, 0, 1], [0, 0, 0, 0, 0, 0, 2, 0, 2], [0, 0, 0, 0, 0, 0, 2, 1, 0], [0, 0, 0, 0, 0, 0, 2, 1, 1], [0, 0, 0, 0, 0, 0, 2, 1, 2], [0, 0, 0, 0, 0, 0, 2, 2, 0], [0, 0, 0, 0, 0, 0, 2, 2, 1], [0, 0, 0, 0, 0, 0, 2, 2, 2], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 2], [0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 1, 0, 1, 2], [0, 0, 0, 0, 0, 1, 0, 2

  0%|          | 0/19683 [00:00<?, ?it/s]

List length: 55
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 2, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0

In [6]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################
config['data']['sparsity'] = nCr(config['data']['n']+config['data']['d'], config['data']['d']) if not laurent else len(list_of_monomial_identifiers)
#nCr(config['data']['n']+config['data']['d'], config['data']['d']) if not laurent else (config['data']['d']*2+1)**config['data']['n'] - np.sum([i for i in range(1, config['data']['d']+1)])#- math.factorial(config['data']['d'])
#nCr(config['data']['n']+config['data']['d'], config['data']['d']) if not laurent else nCr(config['data']['n']+config['data']['d']*2, config['data']['d']*2)

config['data']['sample_sparsity'] = config['data']['sparsity'] if config['data']['sample_sparsity'] == None else config['data']['sample_sparsity']
    
#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['computation'])

initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

<class 'KeyError'>
<class 'KeyError'>


In [7]:
print(path_identifier_polynomial_data)


poly_50000_train_5000_var_9_d_2_laurent_negd_0_prob_0_spars_55_amin_-1_amax_1_xdist_uniform_noise_normal_0.15bmin0.2bmax0.4lowd0.5azero0.25arand0.1_diffX


In [8]:
##############DO NOT CHANGE###################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]
    
print('Variables: ' + str(n) + ' (' + variables + ')')
print('Degree: ' + str(d))
print('Sparsity: ' + str(sparsity)) 
print('Lambda-Net Dataset Size: ' + str(lambda_dataset_size))
print('I-Net Dataset Size: ' + str(polynomial_data_size))
      
print('Coefficient Range: ' + '[' + str(a_min) + ', ' + str(a_max) + ']')
print('Variable Range: ' + '[' + str(x_min) + ', ' + str(x_max) + ']')

Variables: 9 (abcdefghi)
Degree: 2
Sparsity: 55
Lambda-Net Dataset Size: 5000
I-Net Dataset Size: 50000
Coefficient Range: [-1, 1]
Variable Range: [0.05, 1]


# Function Generation

In [9]:
if n==1 and True:
    start = perf_counter()
    # issue with multiprocessing on Windows (backend='sequential' works)
    parallel = Parallel(n_jobs=n_jobs, verbose=0, backend='multiprocessing')
    list_of_polynomials = parallel(delayed(get_polynomial)(border_min, border_max, a_max, a_zero_prob, d, a_random_prob, lower_degree_prob) for i in range(polynomial_data_size))
    list_of_polynomials = np.array(list_of_polynomials).astype('float64')
    end = perf_counter()
    print('Zeit zum generieren:', end-start, 'Sekunden')
    print('Zeit zum generieren pro Funktion:', (end-start)/polynomial_data_size)
else:
    if False:
        list_of_polynomials = np.random.uniform(low=a_min, high=a_max, size=(polynomial_data_size-1000, sparsity))

        list_of_polynomials_2 = np.random.uniform(low=a_min, high=a_max, size=(1000, sparsity))

        list_of_polynomials_2_new = []
        for poly in list_of_polynomials_2:
            poly[-1] = 0
            list_of_polynomials_2_new.append(poly)

        list_of_polynomials = np.vstack([list_of_polynomials, list_of_polynomials_2])
        
    if True:
        list_of_polynomials = np.random.uniform(low=a_min, high=a_max, size=(polynomial_data_size, sparsity))
        
        
    if sample_sparsity < sparsity:
        for polynomial in tqdm(list_of_polynomials):
            sparsity_indices = np.random.choice(nCr(n+d, d), nCr(n+d, d)-sample_sparsity, replace=False)
            for sparsity_index in sparsity_indices:                            
                polynomial[sparsity_index] = 0

list_of_monomial_identifiers_string = [''.join(str(e) for e in monomial_identifier) for monomial_identifier in list_of_monomial_identifiers] if n > 1 else [str(monomial_identifier[0]) for monomial_identifier in list_of_monomial_identifiers]
polynomials_list_df = pd.DataFrame(data=list_of_polynomials, columns=list_of_monomial_identifiers_string).astype('float64')
    
print(len(list_of_monomial_identifiers))
print(polynomials_list_df.shape)

55
(50000, 55)


In [10]:
if same_training_all_lambda_nets:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
    result_list = parallel(delayed(gen_regression_symbolic)(polynomial_array=list_of_polynomials[i], 
                                                               n_samples=lambda_dataset_size,
                                                               noise=noise,
                                                               noise_dist=noise_distrib, 
                                                               seed=RANDOM_SEED, 
                                                               sympy_calculation=False) for i in range(polynomial_data_size))  
else:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
    result_list = parallel(delayed(gen_regression_symbolic)(polynomial_array=list_of_polynomials[i], 
                                                               n_samples=lambda_dataset_size,
                                                               noise=noise,
                                                               noise_dist=noise_distrib, 
                                                               seed=RANDOM_SEED+i, 
                                                               sympy_calculation=False) for i in range(polynomial_data_size))

X_data_list = [[pd.Series(result[0],  index=list_of_monomial_identifiers_string), pd.DataFrame(result[1], columns=list(variables[:n]))] for result in result_list]
y_data_list = [[pd.Series(result[0],  index=list_of_monomial_identifiers_string), pd.DataFrame(result[2], columns=['result'])] for result in result_list]



[Parallel(n_jobs=20)]: Using backend MultiprocessingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   23.1s
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed:  1.0min
[Parallel(n_jobs=20)]: Done 472 tasks      | elapsed:  1.9min
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:  3.0min
[Parallel(n_jobs=20)]: Done 1112 tasks      | elapsed:  4.4min
[Parallel(n_jobs=20)]: Done 1528 tasks      | elapsed:  6.0min
[Parallel(n_jobs=20)]: Done 2008 tasks      | elapsed:  7.8min
[Parallel(n_jobs=20)]: Done 2552 tasks      | elapsed: 10.0min
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed: 12.3min
[Parallel(n_jobs=20)]: Done 3832 tasks      | elapsed: 15.0min
[Parallel(n_jobs=20)]: Done 4568 tasks      | elapsed: 17.8min
[Parallel(n_jobs=20)]: Done 5368 tasks      | elapsed: 20.9min
[Parallel(n_jobs=20)]: Done 6232 tasks      | elapsed: 24.3min
[Parallel(n_jobs=20)]: Done 7160 tasks      | elapsed: 27.9min
[Parallel(n_jobs=20)]: Done 8152 t

In [11]:
X_data_list[0][0]

000000000   -0.251
000000001    0.901
000000002    0.464
000000010    0.197
000000011   -0.688
000000020   -0.688
000000100   -0.884
000000101    0.732
000000110    0.202
000000200    0.416
000001000   -0.959
000001001    0.940
000001010    0.665
000001100   -0.575
000002000   -0.636
000010000   -0.633
000010001   -0.392
000010010    0.050
000010100   -0.136
000011000   -0.418
000020000    0.224
000100000   -0.721
000100001   -0.416
000100010   -0.267
000100100   -0.088
000101000    0.570
000110000   -0.601
000200000    0.028
001000000    0.185
001000001   -0.907
001000010    0.215
001000100   -0.659
001001000   -0.870
001010000    0.898
001100000    0.931
002000000    0.617
010000000   -0.391
010000001   -0.805
010000010    0.368
010000100   -0.120
010001000   -0.756
010010000   -0.010
010100000   -0.931
011000000    0.819
020000000   -0.482
100000000    0.325
100000001   -0.377
100000010    0.040
100000100    0.093
100001000   -0.630
100010000    0.939
100100000    0.550
101000000   

In [12]:
X_data_list[0][1].head()

,a,b,c,d,e,f,g,h,i
0,0.406,0.953,0.745,0.619,0.198,0.198,0.105,0.873,0.621
1,0.723,0.070,0.971,0.841,0.252,0.223,0.224,0.339,0.549
2,0.460,0.327,0.631,0.183,0.328,0.398,0.483,0.796,0.240
3,0.539,0.613,0.094,0.627,0.212,0.112,0.951,0.967,0.818
4,0.339,0.143,0.700,0.468,0.166,0.520,0.083,0.914,0.296


In [13]:
y_data_list[0][0]

000000000   -0.251
000000001    0.901
000000002    0.464
000000010    0.197
000000011   -0.688
000000020   -0.688
000000100   -0.884
000000101    0.732
000000110    0.202
000000200    0.416
000001000   -0.959
000001001    0.940
000001010    0.665
000001100   -0.575
000002000   -0.636
000010000   -0.633
000010001   -0.392
000010010    0.050
000010100   -0.136
000011000   -0.418
000020000    0.224
000100000   -0.721
000100001   -0.416
000100010   -0.267
000100100   -0.088
000101000    0.570
000110000   -0.601
000200000    0.028
001000000    0.185
001000001   -0.907
001000010    0.215
001000100   -0.659
001001000   -0.870
001010000    0.898
001100000    0.931
002000000    0.617
010000000   -0.391
010000001   -0.805
010000010    0.368
010000100   -0.120
010001000   -0.756
010010000   -0.010
010100000   -0.931
011000000    0.819
020000000   -0.482
100000000    0.325
100000001   -0.377
100000010    0.040
100000100    0.093
100001000   -0.630
100010000    0.939
100100000    0.550
101000000   

In [14]:
y_data_list[0][1].head()

,result
0,-0.801
1,1.109
2,-0.556
3,-1.496
4,-0.511


In [15]:
path_polynomials = './data/saved_polynomial_lists/polynomials_sample_' + path_identifier_polynomial_data + '.csv'
polynomials_list_df.to_csv(path_polynomials, index=False)

path_X_data = './data/saved_polynomial_lists/X_sample_' + path_identifier_polynomial_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)#, protocol=2)
    
path_y_data = './data/saved_polynomial_lists/y_sample_' + path_identifier_polynomial_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)#, protocol=2)


In [16]:
counter = 0
for X_data in X_data_list:
    if X_data[0][-1] > 0:
        counter += 1
print(counter/len(X_data_list))
print(counter)

0.5021
25105
